# 아이펠톤에서 사용할 데이터 생성
- annotated data (캐글의 심리학자-환자 대화 셋)의 전체 장문, 상담 받는 사람의 생각, 해당 생각의 인지왜곡 type 과 포멧을 맞춤
- (중국어 -> 영어로 변역된) C2D2 데이터셋의 시나리오+thought (인지 왜곡된 생각)을 맞춰 장문을 만들어냄
- 선행연구 C2d2 dataset: A resource for the cognitive distortion analysis and its impact on mental health. 에서 사용된 것

In [1]:
import os
# from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

In [2]:
# load_dotenv()
os.environ["OPENAI_API_KEY"] = ""

client = OpenAI()

In [3]:
# raw = pd.read_csv("raw_data/Final_Modified_Chunk_3.csv")
raw = pd.read_csv("raw_data/Final_Modified_C2D20911.csv")
raw.head()

,Num,Scenario,Thought,Label
0,1,"I'm an introverted person, and I've just arriv...",Are the people in this environment unfriendly?,Overgeneralization
1,2,"Recently, I feel dizzy sometimes when I stand ...",I'm so dizzy. Am I sick? I should probably go ...,No Distortion
2,3,"I'm walking down the street and feel hungry, b...","I'm tired and there's no place to rest, I'm hu...",Overgeneralization
3,4,"Work has been busy lately, but I have caught a...",Why did I catch a cold at this time? I feel so...,No Distortion
4,5,My mom and I are discussing future plans. She ...,"Mom is trying to control my life again, wantin...",Fortune-telling


In [4]:
# 엑셀 프로그램으로 열었을때 깨지던 부분 확인, 문제 없음 
print(raw["Scenario"][44])
print(raw["Thought"][44])

I have always lived in a family with relatively poor economic conditions and Ir parents do not have a high income.
I am a poor guy. People around me don’t look down on me. I live in pain every day without enough food to eat and warm clothes to wear. I want to die.


In [5]:
# ir 해결? -> GPT에 번역 요청하니 문법적 오류를 알아서 해결해줌 

In [13]:
print(raw["Scenario"][499])
print(raw["Thought"][499])

The takeaway order was stolen.
Who is so wicked! The family can't afford to eat, so they send their children to college just to steal takeout, right?


In [ ]:
def get_chatgpt_response(input_01, input_02):
    # OpenAI API를 통해 ChatGPT에게 한국어로 자연스럽게 다듬어 달라고 요청
    response = client.chat.completions.create(
        model="gpt-4o",
        # model="gpt-4o-mini",
        # model ="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a patient receiving psychological counseling. who can speak english only."},
            {"role": "user", 
             "content": f"다음에 제공되는 2 문장은 너가 쓴 일기인데, 2~3문장이 빠져 있어 {input_01} {input_02}, 주어진 문장을 그대로 변경 없이 포함해서, 비어있는 내용을 영어로 써줘"}
        ]
    )
    # return response['choices'][0]['message']['content']
    return response

In [ ]:
# n개 행만 테스트
result = raw[0:500]
print(result)

In [21]:
def get_chatgpt_response(input_01, input_02, content):
    # OpenAI API를 통해 ChatGPT에게 한국어로 자연스럽게 다듬어 달라고 요청
    response = client.chat.completions.create(
        model="gpt-4o",
        # model="gpt-4o-mini",
        # model ="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a patient receiving psychological counseling. who can speak english only."},
            {"role": "user", 
             "content": content.format(input_01=input_01, input_02=input_02)}
        ]
    )
    # return response['choices'][0]['message']['content']
    return response

In [25]:
# ChatGPT에게 텍스트 다듬기 요청
content = "다음에 제공되는 2 문장은 너가 쓴 일기인데, 2~3문장이 빠져 있어 {input_01} {input_02}, 주어진 문장을 그대로 변경 없이 포함해서, 비어있는 내용을 영어로 써줘"
refined_response = get_chatgpt_response(raw["Scenario"][0], raw["Thought"][0], content)

In [26]:
# 최종 결과 출력
print(refined_response.choices[0].message.content)

I'm an introverted person, and I've just arrived in a new environment where the people around I are mostly unfamiliar. At first, I felt overwhelmed and anxious about making new connections. I wondered, Are the people in this environment unfriendly? However, after a few days, I started to notice some friendly gestures from my new neighbors. This gave me some hope that I can eventually feel comfortable here.


In [24]:
# n개 행만 테스트
result = raw[500:]
print(len(result))
print(result.head())

7003
     Num                                           Scenario  \
500  501  If I fail to complete Ir research project or p...   
501  502  We agreed to go out and play together, but I w...   
502  503             Suddenly a war breaks out where I are.   
503  504   Ir lover passed away unexpectedly some time ago.   
504  505  Someone bullies I and says I are an unimportan...   

                                               Thought                Label  
500  I am afraid that I can't complete the project....  Emotional Reasoning  
501  There must be something urgent that he didn't ...        No Distortion  
502  The war is coming, the sound of artillery fire...  Emotional Reasoning  
503  Nothing is eternal, no one can accompany me to...   Overgeneralization  
504  He probably said that because he's jealous of ...         Mind Reading  


In [35]:
# 반복문
content_00 = "다음에 제공되는 2 문장은 너가 쓴 일기인데, 2~3문장이 빠져 있어 {input_01} {input_02}, 주어진 문장을 그대로 변경 없이 포함해서, 비어있는 내용을 영어로 써줘"
result_list = []
result_00 = result.copy()

# 파일 경로
file_path_list = "data/c2d2_refined01_list.csv"

for i in range(len(result_00)):
    refined_response = get_chatgpt_response(result.iloc[i]["Scenario"], result.iloc[i]["Thought"], content_00)
    result_list.append(refined_response.choices[0].message.content)
        
    # 100번마다 result_list 저장
    if (i + 1) % 100 == 0 or i == len(result) - 1:
        print(f"Saving at iteration {i + 1}")
        
        # result_list를 DataFrame으로 변환
        result_df = pd.DataFrame({"Refined_Thought": result_list})
        
        # 첫 번째 저장은 파일을 새로 생성하고, 이후에는 추가 (append)
        if i + 1 == 100:
            result_df.to_csv(file_path_list, index=False)  # 처음 100개는 새 파일 생성
        else:
            result_df.to_csv(file_path_list, mode='a', header=False, index=False)  # 이후 100개씩 추가

        # 다음 데이터를 위해 result_list 초기화
        result_list = []

# result 불러오기
result_list_all = pd.read_csv(file_path_list)

# 마지막 결과 저장
file_path = "data/c2d2_refined01.csv"

result_00["Refined_Thought"] = result_list_all["Refined_Thought"]
result_00.to_csv(file_path, index=False)

Saving at iteration 100
Saving at iteration 200
Saving at iteration 300
Saving at iteration 400
Saving at iteration 500
Saving at iteration 600
Saving at iteration 700
Saving at iteration 800
Saving at iteration 900
Saving at iteration 1000
Saving at iteration 1100
Saving at iteration 1200
Saving at iteration 1300
Saving at iteration 1400
Saving at iteration 1500
Saving at iteration 1600
Saving at iteration 1700
Saving at iteration 1800
Saving at iteration 1900
Saving at iteration 2000
Saving at iteration 2100
Saving at iteration 2200
Saving at iteration 2300
Saving at iteration 2400
Saving at iteration 2500
Saving at iteration 2600
Saving at iteration 2700
Saving at iteration 2800
Saving at iteration 2900
Saving at iteration 3000
Saving at iteration 3100
Saving at iteration 3200
Saving at iteration 3300
Saving at iteration 3400
Saving at iteration 3500
Saving at iteration 3600
Saving at iteration 3700
Saving at iteration 3800
Saving at iteration 3900
Saving at iteration 4000
Saving at

ValueError: Length of values (3) does not match length of index (7003)

### content를 나눠서 여러개 실행해보기
하다 말았음

In [80]:
# 반복문
content_01 = "다음에 제공되는 2 문장은 너가 쓴 일기인데, 2~3문장이 빠져 있어 {input_01} {input_02}, 주어진 문장을 그대로 변경 없이 포함해서, 비어있는 내용을 영어로 써줘"
result_list = []
result_01 = result.copy()

for i in range(len(result)):    
    refined_response = get_chatgpt_response(result["Scenario"][i], result["Thought"][i], content_01)
    result_list.append(refined_response.choices[0].message.content)
    
# 결과 합침
result_01["Refined_Thought"] = result_list

# 저장
result_01.to_csv("data/c2d2_refined01_0_20.csv", index=False)

In [81]:
# 나중에 성과가 안 나면, content를 바꿔서 다시 해보기 - '나는 인지왜곡 데이터를 만드는 중이야~ label에 맞춰서 빈 내용을 채워줘~'같이 해보면 될 듯 
# 코드는 만들다 말았음 

def get_chatgpt_response_wlabel(input_01, input_02, label, content):
    # OpenAI API를 통해 ChatGPT에게 한국어로 자연스럽게 다듬어 달라고 요청
    response = client.chat.completions.create(
        model="gpt-4o",
        # model="gpt-4o-mini",
        # model ="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a patient receiving psychological counseling. who can speak english only."},
            {"role": "user", 
             "content": content.format(input_01=input_01, input_02=input_02, label = label)}
        ]
    )
    # return response['choices'][0]['message']['content']
    return response

# 반복문
content_02 = "다음에 제공되는 2 문장은 너가 쓴 일기인데, 2~3문장이 빠져 있어 {input_01} {input_02}, 주어진 문장을 그대로 변경 없이 포함해서, 비어있는 내용을 영어로 써줘"
result_list = []
result_02 = result.copy()

for i in range(len(result)):    
    refined_response = get_chatgpt_response(result["Scenario"][i], result["Thought"][i], content_02)
    result_list.append(refined_response.choices[0].message.content)
    
# 결과 합침
result_02["Refined_Thought"] = result_list

# 저장
result_02.to_csv("data/c2d2_refined02_0_20.csv", index=False)

C:\Users\saink\AppData\Local\Temp\ipykernel_161576\2848543506.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["Refined_Thought"] = result_list


In [82]:
# 반복문
content_03 = "다음에 제공되는 2 문장은 너가 쓴 일기인데, 2~3문장이 빠져 있어 {input_01} {input_02}, 주어진 문장을 그대로 변경 없이 포함해서, 비어있는 내용을 영어로 써줘"
result_list = []
result_03 = result.copy()

for i in range(len(result)):    
    refined_response = get_chatgpt_response(result["Scenario"][i], result["Thought"][i], content_03)
    result_list.append(refined_response.choices[0].message.content)
    
# 결과 합침
result_03["Refined_Thought"] = result_list

# 저장
result_03.to_csv("data/c2d2_refined03_0_20.csv", index=False)

,Num,Scenario,Thought,Label,Refined_Thought
0,1,"I'm an introverted person, and I've just arriv...",Are the people in this environment unfriendly?,Overgeneralization,"Sure, I can help with that. Here's a possible ..."
1,2,"Recently, I feel dizzy sometimes when I stand ...",I'm so dizzy. Am I sick? I should probably go ...,No Distortion,"Recently, I feel dizzy sometimes when I stand ..."
2,3,"I'm walking down the street and feel hungry, b...","I'm tired and there's no place to rest, I'm hu...",Overgeneralization,"I'm walking down the street and feel hungry, b..."
3,4,"Work has been busy lately, but I have caught a...",Why did I catch a cold at this time? I feel so...,No Distortion,"Work has been busy lately, but I have caught a..."
4,5,My mom and I are discussing future plans. She ...,"Mom is trying to control my life again, wantin...",Fortune-telling,My mom and I are discussing future plans. She ...
...,...,...,...,...,...
495,496,Ir parents are dissatisfied with I because I h...,My parents' dissatisfaction with me is all due...,Personalization,"Sure, I can help with that. Here is the comple..."
496,497,"When someone says I look unhappy, it's as if t...",I didn't know I was this kind of person. No wo...,Labeling,"When someone says I look unhappy, it's as if t..."
497,498,I are pushed to the point of explosion by Ir f...,I have worked so hard for this family! But you...,All-or-nothing thinking,"I came home after a long day at work, only to ..."
498,499,I were preparing to run for president of the s...,It turns out that I was too self-righteous. I ...,Labeling,Here is the completed diary entry with the mis...


# 데이터 정제
### GPT의 응답에 원하지 않는 GPT의 의견이 들어가 있음

In [85]:
# 데이터 불러와서, 정제 
raw_2 = pd.read_csv("data/c2d2_refined_0_500.csv")
raw_2.head()

,Num,Scenario,Thought,Label,Refined_Thought
0,1,"I'm an introverted person, and I've just arriv...",Are the people in this environment unfriendly?,Overgeneralization,"Sure, I can help with that. Here's a possible ..."
1,2,"Recently, I feel dizzy sometimes when I stand ...",I'm so dizzy. Am I sick? I should probably go ...,No Distortion,"Recently, I feel dizzy sometimes when I stand ..."
2,3,"I'm walking down the street and feel hungry, b...","I'm tired and there's no place to rest, I'm hu...",Overgeneralization,"I'm walking down the street and feel hungry, b..."
3,4,"Work has been busy lately, but I have caught a...",Why did I catch a cold at this time? I feel so...,No Distortion,"Work has been busy lately, but I have caught a..."
4,5,My mom and I are discussing future plans. She ...,"Mom is trying to control my life again, wantin...",Fortune-telling,My mom and I are discussing future plans. She ...


In [154]:
# t = raw_2["Refined_Thought"][0]
# t = raw_2["Refined_Thought"][6]
# t = raw_2["Refined_Thought"][18]
# t = raw_2["Refined_Thought"][21]
# t = raw_2["Refined_Thought"][114]
# t = raw_2["Refined_Thought"][364]
# t = raw_2["Refined_Thought"][409]
# t = raw_2["Refined_Thought"][30]
# t = raw_2["Refined_Thought"][62]
t = raw_2["Refined_Thought"][461]
t

"Of course. Here are two possible completions for your diary entry using the given sentence exactly as it is:\n\n---\n\nI was talking to someone, and their phone rings, but they don’t pick up. He might not have heard it. Maybe it’s not important, or perhaps they’re just too busy to answer right now. I hope it's nothing urgent.\n\n---\n\nYesterday at the coffee shop, I was talking to someone and their phone rings, but they don't pick up. He might not have heard it. After a while, I wondered if it was someone important calling, but I didn't want to pry. It made me think about how often we ignore calls nowadays.\n\n---\n\nI hope these examples help!"

In [155]:
x = t.split("\n")
try:
    print("len",len(x),"0:",x[0],"1",x[1],"2",x[2],"3",x[3],"4",x[4])
except:
    try:   
        print("len",len(x),"0:",x[0],"1",x[1],"2",x[2],"3",x[3])
    except:
        print("len",len(x),"0:",x[0],"1",x[1],"2",x[2])    

len 13 0: Of course. Here are two possible completions for your diary entry using the given sentence exactly as it is: 1  2 --- 3  4 I was talking to someone, and their phone rings, but they don’t pick up. He might not have heard it. Maybe it’s not important, or perhaps they’re just too busy to answer right now. I hope it's nothing urgent.


In [156]:
# 텍스트 정재

def clean_refined_thought(text):
    parts = text.split('\n')
    # 챗봇이 말한부분 + 본문 으로 되어 있는 경우
    if len(parts) == 3:
        rt =  parts[2]
    elif len(parts) == 4 and "--" in parts[2]:
        rt = parts[3]
    elif len(parts) == 5 and "Diary Entry" in parts[2]:
        rt = parts[4]
    elif len(parts) == 5 and "--" in parts[2]:
        rt = parts[4]
    elif len(parts) == 5 and "--" not in parts[2]:
        rt = parts[2]
    elif len(parts) == 7 and "Option" in parts[2]:
        rt = parts[3]
    elif len(parts) >= 7 and "--" in parts[2]:
        rt = parts[4]
    # elif len(parts) == 9, 13 and "--" in parts[2]:
        # rt = parts[4]
    else: 
        rt = text
    rt = rt.replace('"','') # 큰 따옴표 제거
    return rt

# Apply the cleaning function to the 'Refined_Thought' column
raw_2['Cleaned_Refined_Thought'] = raw_2['Refined_Thought'].apply(clean_refined_thought)

raw_2.head()

,Num,Scenario,Thought,Label,Refined_Thought,Cleaned_Refined_Thought
0,1,"I'm an introverted person, and I've just arriv...",Are the people in this environment unfriendly?,Overgeneralization,"Sure, I can help with that. Here's a possible ...","I'm an introverted person, and I've just arriv..."
1,2,"Recently, I feel dizzy sometimes when I stand ...",I'm so dizzy. Am I sick? I should probably go ...,No Distortion,"Recently, I feel dizzy sometimes when I stand ...","Recently, I feel dizzy sometimes when I stand ..."
2,3,"I'm walking down the street and feel hungry, b...","I'm tired and there's no place to rest, I'm hu...",Overgeneralization,"I'm walking down the street and feel hungry, b...","I'm walking down the street and feel hungry, b..."
3,4,"Work has been busy lately, but I have caught a...",Why did I catch a cold at this time? I feel so...,No Distortion,"Work has been busy lately, but I have caught a...","Work has been busy lately, but I have caught a..."
4,5,My mom and I are discussing future plans. She ...,"Mom is trying to control my life again, wantin...",Fortune-telling,My mom and I are discussing future plans. She ...,My mom and I are discussing future plans. She ...


In [157]:
raw_2.to_csv("data/c2d2_refined_5000_cleaned.csv", index=False)

In [ ]:
# 데이터 합쳐서 정제
